In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor



data = pd.read_csv('/content/sample_data/train.csv')
data.dropna(axis=0, subset=['revenue'], inplace=True)
train_y = data.revenue
train_X = data.drop(['revenue'], axis=1).select_dtypes(exclude=['object'])

test_data = pd.read_csv('/content/sample_data/test.csv')
test_data.dropna(axis=0, subset=['revenue'], inplace=True)
test_y = test_data.revenue
test_X = test_data.drop(['revenue'], axis=1).select_dtypes(exclude=['object'])


In [2]:
train_X.head()

,budget,popularity,runtime,vote_average,vote_count,year,day_1,day_2,day_3,day_4,day_5,day_6,day_7,day_8,day_9,day_10,day_11,day_12,day_13,day_14,day_15,day_16,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25,day_26,day_27,day_28,day_29,day_30,day_31,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,day_name_Friday,day_name_Monday,day_name_Saturday,day_name_Sunday,day_name_Thursday,day_name_Tuesday,day_name_Wednesday,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,30000000.0,21.946943,81.0,7.7,5415.0,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,65000000.0,17.015539,104.0,6.9,2413.0,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
2,16000000.0,3.859495,127.0,6.1,34.0,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,60000000.0,17.924927,170.0,7.7,1886.0,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0
4,35000000.0,5.231580,106.0,5.5,174.0,1995.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [0]:
train_y.head()

0    373554033.0
1    262797249.0
2     81452156.0
3    187436818.0
4     64350171.0
Name: revenue, dtype: float64

In [0]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=True)


XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [0]:
predictions = my_model.predict(test_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))


Mean Absolute Error : 88770876.50368923


In [0]:
my_model = XGBRegressor(n_estimators=100000, learning_rate = 0.01)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(test_X, test_y)], verbose=False)


XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.01, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [0]:
predictions = my_model.predict(test_X)

print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))


Mean Absolute Error : 85227186.07817926


In [0]:
from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
clf_xgb = XGBRegressor(objective = 'reg:squarederror')
param_dist = {
              'learning_rate': [0.001, 0.01, 0.065, 0.1], #so called `eta` value
              'max_depth': [x for x in range(5,15)],
              'min_child_weight': [x for x in range(5,10)],
              'subsample': [x/10 for x in range(0,10)],
              'colsample_bytree': [0.1,0.2,0.3,0.4],
              'n_estimators': [500,1000,1500], #number of trees, change it to 1000 for better results
             }


clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = 10,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'neg_mean_absolute_error', 
                         verbose = 3, 
                         n_jobs = -1)
clf.fit(train_X, train_y)

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  2.9min finished


RandomizedSearchCV(cv=10, error_score=0,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, gamma=None,
                                          gpu_id=None, importance_type='gain',
                                          interaction_constraints=None,
                                          learning_rate=None,
                                          max_delta_step=None, max_depth=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_e...
                   param_distributions={'colsample_bytree': [0.1, 0.2, 0.3,
                                                             0.4],
                                        'learni

In [0]:
print("best params {}".format(clf.best_params_))
print("best_score {}".format(clf.best_score_))

predictions = clf.predict(test_X)
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))




best params {'subsample': 0.5, 'n_estimators': 1000, 'min_child_weight': 0, 'max_depth': 8, 'learning_rate': 0.065, 'colsample_bytree': 0.2}
best_score -44671110.689359896
Mean Absolute Error : 107392076.72835286


In [0]:
clf_xgb = XGBRegressor(objective = 'reg:squarederror')
param_dist = {
              'learning_rate': [0.05, 0.065, 0.1], #so called `eta` value
              'max_depth': [x for x in range(5,15)],
              'min_child_weight': [0,.5,.7,1],
              'subsample': [.5,.7,.9],
              'colsample_bytree': [0.2,0.4,.8],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
             }


clf = RandomizedSearchCV(clf_xgb, 
                         param_distributions = param_dist,
                         cv = 10,  
                         n_iter = 5, # you want 5 here not 25 if I understand you correctly 
                         scoring = 'neg_mean_absolute_error', 
                         verbose = 3, 
                         n_jobs = -1)
clf.fit(train_X, train_y)
print("best params {}".format(clf.best_params_))
print("best_score {}".format(clf.best_score_))

predictions = clf.predict(test_X)
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))


Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  5.9min finished


best params {'subsample': 0.9, 'n_estimators': 1000, 'min_child_weight': 0.7, 'max_depth': 6, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
best_score -39136644.58696466
Mean Absolute Error : 86374420.86783855


In [0]:
clf.cv_results_

{'mean_fit_time': array([14.52140152, 58.53682318,  9.46899335, 33.66344225, 24.40438671]),
 'std_fit_time': array([0.23464231, 0.74953392, 0.06198519, 0.39369309, 0.8221211 ]),
 'mean_score_time': array([0.1046505 , 0.2481389 , 0.03993444, 0.08871601, 0.05125177]),
 'std_score_time': array([0.01847304, 0.03351931, 0.00129701, 0.00239338, 0.00475491]),
 'param_subsample': masked_array(data=[0.7, 0.5, 0.7, 0.9, 0.9],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1000, 1000, 1000, 1000, 1000],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_child_weight': masked_array(data=[0, 1, 0.5, 0.7, 0.7],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[8, 12, 5, 8, 6],
              mask=[False, False, False, False, False],
   

In [0]:
from sklearn.model_selection import GridSearchCV
xgb_model = XGBRegressor(pbjective = 'reg:squarederror')

parameters = { #when use hyperthread, xgboost may become slower
              'learning_rate': [0.001, 0.05], #so called `eta` value
              'max_depth': [6,9],
              'min_child_weight': [.6,.7],
              'subsample': [.9,1],
              'colsample_bytree': [.8,.9],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
             }

clf_grid = GridSearchCV(xgb_model, parameters, n_jobs=-1,  
                   scoring='neg_mean_absolute_error',
                   cv = 10,
                   verbose=2, refit=True)

clf_grid.fit(train_X, train_y)

#trust your CV!
#best_parameters, score, _ = max(clf.cv_results_, key=lambda x: x[1])
#print('Raw AUC score:', score)
8
#for param_name in sorted(best_parameters.keys()):
 #   print("%s: %r" % (param_name, best_parameters[param_name]))
print("best params {}".format(clf_grid.best_params_))
print("best_score {}".format(clf_grid.best_score_))


Fitting 10 folds for each of 32 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed: 46.1min finished


best params {'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 0.6, 'n_estimators': 1000, 'subsample': 0.9}
best_score -38998537.9328269


In [0]:
predictions = clf_grid.predict(test_X)
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, test_y)))



Mean Absolute Error : 87179043.39409722


In [0]:
new_model = XGBRegressor(n_estimators=100000, learning_rate = 0.05,colsample_bytree = 0.9, max_depth = 6, min_child_weight = .6 , subsample = .9)
frames = [train_X, test_X]
newTrain = pd.concat(frames)
frames2 = [train_y, test_y]
newTest = pd.concat(frames2)

new_model.fit(newTrain, newTest, early_stopping_rounds=10, 
             eval_set=[(test_X, test_y)], verbose=2)
guesses = new_model.predict(test_X)
print("Mean Absolute Error : " + str(mean_absolute_error(guesses, test_y)))


[0]	validation_0-rmse:363895520.00000
Will train until validation_0-rmse hasn't improved in 10 rounds.
[2]	validation_0-rmse:337466176.00000
[4]	validation_0-rmse:311491104.00000
[6]	validation_0-rmse:288864000.00000
[8]	validation_0-rmse:268787232.00000
[10]	validation_0-rmse:249459936.00000
[12]	validation_0-rmse:233942192.00000
[14]	validation_0-rmse:217062064.00000
[16]	validation_0-rmse:202024128.00000
[18]	validation_0-rmse:188432176.00000
[20]	validation_0-rmse:176696704.00000
[22]	validation_0-rmse:165882048.00000
[24]	validation_0-rmse:155541200.00000
[26]	validation_0-rmse:145717552.00000
[28]	validation_0-rmse:136750160.00000
[30]	validation_0-rmse:129924864.00000
[32]	validation_0-rmse:123746720.00000
[34]	validation_0-rmse:117795016.00000
[36]	validation_0-rmse:112478368.00000
[38]	validation_0-rmse:106874904.00000
[40]	validation_0-rmse:102339984.00000
[42]	validation_0-rmse:98132088.00000
[44]	validation_0-rmse:94761768.00000
[46]	validation_0-rmse:91497600.00000
[48]	va

[420]	validation_0-rmse:21478986.00000
[422]	validation_0-rmse:21390936.00000
[424]	validation_0-rmse:21329748.00000
[426]	validation_0-rmse:21223498.00000
[428]	validation_0-rmse:21157376.00000
[430]	validation_0-rmse:21082028.00000
[432]	validation_0-rmse:21030112.00000
[434]	validation_0-rmse:20864602.00000
[436]	validation_0-rmse:20763126.00000
[438]	validation_0-rmse:20709556.00000
[440]	validation_0-rmse:20559056.00000
[442]	validation_0-rmse:20384500.00000
[444]	validation_0-rmse:20301294.00000
[446]	validation_0-rmse:20257774.00000
[448]	validation_0-rmse:20243856.00000
[450]	validation_0-rmse:20167154.00000
[452]	validation_0-rmse:20125698.00000
[454]	validation_0-rmse:20060512.00000
[456]	validation_0-rmse:19915692.00000
[458]	validation_0-rmse:19893682.00000
[460]	validation_0-rmse:19861400.00000
[462]	validation_0-rmse:19808732.00000
[464]	validation_0-rmse:19760934.00000
[466]	validation_0-rmse:19740490.00000
[468]	validation_0-rmse:19620230.00000
[470]	validation_0-rmse:1

[842]	validation_0-rmse:10808153.00000
[844]	validation_0-rmse:10774128.00000
[846]	validation_0-rmse:10715442.00000
[848]	validation_0-rmse:10677009.00000
[850]	validation_0-rmse:10621754.00000
[852]	validation_0-rmse:10536855.00000
[854]	validation_0-rmse:10516355.00000
[856]	validation_0-rmse:10502686.00000
[858]	validation_0-rmse:10486958.00000
[860]	validation_0-rmse:10415868.00000
[862]	validation_0-rmse:10411343.00000
[864]	validation_0-rmse:10411648.00000
[866]	validation_0-rmse:10373728.00000
[868]	validation_0-rmse:10342546.00000
[870]	validation_0-rmse:10318104.00000
[872]	validation_0-rmse:10288974.00000
[874]	validation_0-rmse:10204497.00000
[876]	validation_0-rmse:10177041.00000
[878]	validation_0-rmse:10138064.00000
[880]	validation_0-rmse:10114909.00000
[882]	validation_0-rmse:10080740.00000
[884]	validation_0-rmse:10034786.00000
[886]	validation_0-rmse:10028749.00000
[888]	validation_0-rmse:9980940.00000
[890]	validation_0-rmse:9950460.00000
[892]	validation_0-rmse:992

[1266]	validation_0-rmse:6001546.00000
[1268]	validation_0-rmse:5992406.00000
[1270]	validation_0-rmse:5978337.50000
[1272]	validation_0-rmse:5977156.50000
[1274]	validation_0-rmse:5948473.50000
[1276]	validation_0-rmse:5922654.00000
[1278]	validation_0-rmse:5899518.00000
[1280]	validation_0-rmse:5891078.00000
[1282]	validation_0-rmse:5881158.50000
[1284]	validation_0-rmse:5861013.00000
[1286]	validation_0-rmse:5851230.50000
[1288]	validation_0-rmse:5847977.50000
[1290]	validation_0-rmse:5829627.50000
[1292]	validation_0-rmse:5822555.00000
[1294]	validation_0-rmse:5803788.50000
[1296]	validation_0-rmse:5789477.00000
[1298]	validation_0-rmse:5772895.00000
[1300]	validation_0-rmse:5764209.50000
[1302]	validation_0-rmse:5761718.50000
[1304]	validation_0-rmse:5751407.50000
[1306]	validation_0-rmse:5745099.00000
[1308]	validation_0-rmse:5714739.50000
[1310]	validation_0-rmse:5682077.00000
[1312]	validation_0-rmse:5666782.50000
[1314]	validation_0-rmse:5656741.50000
[1316]	validation_0-rmse:

[1688]	validation_0-rmse:3619968.50000
[1690]	validation_0-rmse:3611644.75000
[1692]	validation_0-rmse:3611106.00000
[1694]	validation_0-rmse:3590050.50000
[1696]	validation_0-rmse:3583397.75000
[1698]	validation_0-rmse:3573286.50000
[1700]	validation_0-rmse:3566662.00000
[1702]	validation_0-rmse:3561605.00000
[1704]	validation_0-rmse:3536063.00000
[1706]	validation_0-rmse:3524476.25000
[1708]	validation_0-rmse:3517707.25000
[1710]	validation_0-rmse:3509797.50000
[1712]	validation_0-rmse:3498176.75000
[1714]	validation_0-rmse:3485204.75000
[1716]	validation_0-rmse:3482952.75000
[1718]	validation_0-rmse:3474022.50000
[1720]	validation_0-rmse:3465981.00000
[1722]	validation_0-rmse:3459356.75000
[1724]	validation_0-rmse:3452327.50000
[1726]	validation_0-rmse:3444960.75000
[1728]	validation_0-rmse:3441348.00000
[1730]	validation_0-rmse:3432971.00000
[1732]	validation_0-rmse:3424098.25000
[1734]	validation_0-rmse:3418497.50000
[1736]	validation_0-rmse:3411010.25000
[1738]	validation_0-rmse:

...

[9120]	validation_0-rmse:4955.06348
[9122]	validation_0-rmse:4956.35742
[9124]	validation_0-rmse:4955.31299
[9126]	validation_0-rmse:4952.64062
[9128]	validation_0-rmse:4950.82568
[9130]	validation_0-rmse:4943.67285
[9132]	validation_0-rmse:4936.44971
[9134]	validation_0-rmse:4930.21777
[9136]	validation_0-rmse:4929.14111
[9138]	validation_0-rmse:4928.35742
[9140]	validation_0-rmse:4921.86914
[9142]	validation_0-rmse:4911.41943
[9144]	validation_0-rmse:4911.64404
[9146]	validation_0-rmse:4906.73047
[9148]	validation_0-rmse:4906.44678
[9150]	validation_0-rmse:4894.04932
[9152]	validation_0-rmse:4883.35693
[9154]	validation_0-rmse:4882.64990
[9156]	validation_0-rmse:4869.03955
[9158]	validation_0-rmse:4864.69775
[9160]	validation_0-rmse:4853.20752
[9162]	validation_0-rmse:4844.83496
[9164]	validation_0-rmse:4846.57910
[9166]	validation_0-rmse:4845.74121
[9168]	validation_0-rmse:4831.05127
[9170]	validation_0-rmse:4826.49512
[9172]	validation_0-rmse:4794.47949
[9174]	validation_0-rmse:478

[9576]	validation_0-rmse:3333.37256
[9578]	validation_0-rmse:3329.92481
[9580]	validation_0-rmse:3323.39453
[9582]	validation_0-rmse:3318.00049
[9584]	validation_0-rmse:3317.77026
[9586]	validation_0-rmse:3308.78101
[9588]	validation_0-rmse:3304.83691
[9590]	validation_0-rmse:3304.78687
[9592]	validation_0-rmse:3299.98975
[9594]	validation_0-rmse:3292.03760
[9596]	validation_0-rmse:3284.39648
[9598]	validation_0-rmse:3284.27954
[9600]	validation_0-rmse:3280.63232
[9602]	validation_0-rmse:3274.90747
[9604]	validation_0-rmse:3271.73071
[9606]	validation_0-rmse:3262.13892
[9608]	validation_0-rmse:3261.22412
[9610]	validation_0-rmse:3248.28760
[9612]	validation_0-rmse:3246.23144
[9614]	validation_0-rmse:3240.26831
[9616]	validation_0-rmse:3238.55225
[9618]	validation_0-rmse:3237.16846
[9620]	validation_0-rmse:3231.97681
[9622]	validation_0-rmse:3228.43262
[9624]	validation_0-rmse:3211.34106
[9626]	validation_0-rmse:3210.68237
[9628]	validation_0-rmse:3202.98877
[9630]	validation_0-rmse:320

[10032]	validation_0-rmse:2302.53589
[10034]	validation_0-rmse:2297.08032
[10036]	validation_0-rmse:2292.62817
[10038]	validation_0-rmse:2291.28271
[10040]	validation_0-rmse:2287.02124
[10042]	validation_0-rmse:2282.13232
[10044]	validation_0-rmse:2280.13794
[10046]	validation_0-rmse:2273.24707
[10048]	validation_0-rmse:2265.82935
[10050]	validation_0-rmse:2258.97510
[10052]	validation_0-rmse:2257.56958
[10054]	validation_0-rmse:2255.98901
[10056]	validation_0-rmse:2253.63574
[10058]	validation_0-rmse:2253.62109
[10060]	validation_0-rmse:2252.66992
[10062]	validation_0-rmse:2250.23291
[10064]	validation_0-rmse:2245.40063
[10066]	validation_0-rmse:2241.31201
[10068]	validation_0-rmse:2238.87695
[10070]	validation_0-rmse:2239.21826
[10072]	validation_0-rmse:2238.98218
[10074]	validation_0-rmse:2237.27881
[10076]	validation_0-rmse:2227.22681
[10078]	validation_0-rmse:2225.25049
[10080]	validation_0-rmse:2223.12476
[10082]	validation_0-rmse:2220.63232
[10084]	validation_0-rmse:2216.12671
[

[10476]	validation_0-rmse:1611.99390
[10478]	validation_0-rmse:1610.35364
[10480]	validation_0-rmse:1610.07666
[10482]	validation_0-rmse:1607.54626
[10484]	validation_0-rmse:1601.47156
[10486]	validation_0-rmse:1600.39648
[10488]	validation_0-rmse:1596.80469
[10490]	validation_0-rmse:1587.29724
[10492]	validation_0-rmse:1588.44495
[10494]	validation_0-rmse:1586.65088
[10496]	validation_0-rmse:1577.11462
[10498]	validation_0-rmse:1568.98755
[10500]	validation_0-rmse:1568.92761
[10502]	validation_0-rmse:1564.98669
[10504]	validation_0-rmse:1561.74316
[10506]	validation_0-rmse:1558.67102
[10508]	validation_0-rmse:1557.35803
[10510]	validation_0-rmse:1555.47681
[10512]	validation_0-rmse:1556.10730
[10514]	validation_0-rmse:1553.38635
[10516]	validation_0-rmse:1552.09387
[10518]	validation_0-rmse:1548.23975
[10520]	validation_0-rmse:1545.69470
[10522]	validation_0-rmse:1538.45960
[10524]	validation_0-rmse:1537.60303
[10526]	validation_0-rmse:1536.24756
[10528]	validation_0-rmse:1534.15478
[

[10920]	validation_0-rmse:1110.20923
[10922]	validation_0-rmse:1109.34326
[10924]	validation_0-rmse:1105.91858
[10926]	validation_0-rmse:1105.82080
[10928]	validation_0-rmse:1105.74451
[10930]	validation_0-rmse:1102.95630
[10932]	validation_0-rmse:1100.76856
[10934]	validation_0-rmse:1098.29883
[10936]	validation_0-rmse:1098.40637
[10938]	validation_0-rmse:1095.41394
[10940]	validation_0-rmse:1094.44385
[10942]	validation_0-rmse:1094.27539
[10944]	validation_0-rmse:1093.52527
[10946]	validation_0-rmse:1092.80774
[10948]	validation_0-rmse:1092.46851
[10950]	validation_0-rmse:1091.56323
[10952]	validation_0-rmse:1090.26794
[10954]	validation_0-rmse:1088.70264
[10956]	validation_0-rmse:1088.01196
[10958]	validation_0-rmse:1086.99072
[10960]	validation_0-rmse:1085.69690
[10962]	validation_0-rmse:1084.61243
[10964]	validation_0-rmse:1083.22705
[10966]	validation_0-rmse:1083.18799
[10968]	validation_0-rmse:1078.92615
[10970]	validation_0-rmse:1076.99695
[10972]	validation_0-rmse:1075.50574
[

In [3]:
from sklearn.metrics import mean_absolute_error

new_model2 = XGBRegressor(n_estimators=10000, learning_rate = 0.01,colsample_bytree = 0.9, max_depth = 6, min_child_weight = .6 , subsample = .9)

new_model2.fit(train_X, train_y, early_stopping_rounds=10, 
             eval_set=[(train_X, train_y)], verbose=False, eval_metric = 'mae')
guesses = new_model2.predict(test_X)
print("Mean Absolute Error : " + str(mean_absolute_error(guesses, test_y)))


[19:30:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error : 84341698.47424316


In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(new_model2, test_X, test_y, scoring='r2') 

In [8]:
Y_of_train = new_model2.predict(train_X)
score = r2_score(train_y, Y_of_train)
print(score)



0.9996052254863531


In [5]:
from sklearn.metrics import r2_score
score = r2_score(test_y, guesses)
print(score)


0.7788406731473005
